In [5]:
# Imports
from Tunes_cleaned import tunes as raw_tunes
import Tune_Cleaning
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import importlib
%matplotlib inline

In [15]:
importlib.reload(Tune_Cleaning)
pass

In [2]:
tunes = pd.DataFrame.from_dict(raw_tunes, orient='index')
print("{} Songs imported".format(tunes.shape[0]))

30840 Songs imported


In [16]:
tunes_clean = Tune_Cleaning.filter_tunes(tunes)
print("After Cleaning {} songs remain".format(tunes.shape[0]))
tunes_clean.head(5)

After Cleaning 30840 songs remain


,mode,type,tune,setting,meter,abc
0,Edorian,reel,1,1,4/4,|:D2|EBBAB2EB|B2ABdBAG|FDADBDAD|FDADdAFD|EBBAB...
1,Dmajor,reel,2,2,4/4,|A2FAA2dB|A2FABEE2|A2FAA2Bd|egfdedBd|A2FAA2dB|...
2,Dmajor,hornpipe,4,4,4/4,|:ag|faefdeAF|DFAdf2ef|gbegdeAF|GBAGE2ag|faefd...
3,Gmajor,reel,8,8,4/4,|:G2GDEDEG|AGABd2Bd|egedBAGA|BAGEEDDE|G2GDEDEG...
4,Amajor,reel,11,11,4/4,|:ABcdeAce|fedfedcB|ABcdeAce|1feBcd2cB:|2feBcd...


In [20]:
tunes_clean = Tune_Cleaning.clean_tunes(tunes_clean)
tunes_clean.head(5)

,mode,type,tune,setting,meter,abc
0,Edorian,reel,1,1,4/4,|D2|EBBAB2EB|B2ABdBAG|FDADBDAD|FDADdAFD|EBBAB2...
1,Dmajor,reel,2,2,4/4,|A2FAA2dB|A2FABEE2|A2FAA2Bd|egfdedBd|A2FAA2dB|...
2,Dmajor,hornpipe,4,4,4/4,|ag|faefdeAF|DFAdf2ef|gbegdeAF|GBAGE2ag|faefde...
3,Gmajor,reel,8,8,4/4,|G2GDEDEG|AGABd2Bd|egedBAGA|BAGEEDDE|G2GDEDEG|...
4,Amajor,reel,11,11,4/4,|ABcdeAce|fedfedcB|ABcdeAce|1feBcd2cB|2feBcdef...


In [21]:
def vectorize_tunes(frame):
    """
    given a frame with an 'abc' column, it creates two new columns, 'abc_unfolded', 'timing_vector' and 'abc_vectorized'
    abc_unfolded is abc notation but with timing data made explicit (each note representing a 16th note)
    timing_vector is a vector where 0s represent distinct notes and 1s represent a continued note from the previous step
    abc_vectorized is the same as abc_unfolded but with the letters converted into numbers
    """
    pass

In [30]:
def unfold_abc(abc_string):
    """
    given an abc list returns a tuple of 2 lists: notes and holds
    """
    abc_str=""
    tunevec=np.zeros()
    print(abc_string)

In [23]:
def unfold_bar():
    """
    given a bar of abc notation it expands each note to represent timing information explicitly
    """
    pass

In [ ]:
def expand_bar(bar):
    """
    Takes a str representing a single bar of notes (no barlines)
    Returns a string with each 16th note as it's own character
    Note: because of octaves and other random noise bars are NOT guaranteed to be 16 characters
    """
    output = ""
    if len(bar)==0:
        return output
    if bar[0].isnumeric():
        bar = bar[1:]
    #if the bar contains only letters then all letters are of equal length
    if bar.isalpha():
        time_value = 16//len(bar)
        for char in bar:
            output+= char*time_value
        return output
    
    # if the bar contains more timing information, we must parse it
    
    # === Long Notes ===
    # long notes are notes with a number following them, c2 is twice the length of c
    # long notes regex group
    long_notes_re = re.compile('([a-zA-Z],*\'*(\d))')
    long_notes = long_notes_re.findall(bar)
    ### inline func to replace long notes
    def replace_long_notes(note):
        """
        Takes a re match and replaces it with \d occurences of the letter
        """
        str = note.group()[:-1]
        n = int(note.group(2))
        return str*n
    ### endfunc
    if long_notes:
        bar = long_notes_re.sub(replace_long_notes,bar)
        
    # === SHORT NOTES ===
    # short notes are notes with a / and a number, c/2 is half the length of c
    # short notes regex group, group 2 matches a number if it is present
    highest_mult = 2 #default for / in ABC notation is to halve the length
    short_notes_re = re.compile('([a-zA-Z],*\'*)(/)(\d*)')
    short_notes = short_notes_re.findall(bar)
    if short_notes:
        # check if any of the note divisions are higher than the default of 2
        for note, slash, num in short_notes:
            if num.isnumeric and num is not "":
#                 print("{} {} {} {}".format(note,slash,type(num),type(highest_mult)))
                if int(num)>int(highest_mult):
                    highest_mult=int(num)
    # get every non-short note and multiply it by the highest multiple value
    not_short_notes_re = re.compile('[a-zA-Z],*(?!/)')
    not_short_notes = not_short_notes_re.findall(bar)    
    ### inline function to replace notes
    def lengthen_non_short_notes(match_str):
        return match_str.group()*highest_mult
    ### endfunc
    # lengthen all notes that aren't 'short notes'
    bar = not_short_notes_re.sub(lengthen_non_short_notes, bar)
    def replace_short_notation(match_str):
        return match_str.group(1)
    bar = short_notes_re.sub(replace_short_notation, bar)
    #lastly some bars need to be expanded to exactly 16 notes
    #note: this won't fix octave characters as they correspond to 2 characters right now, the same test should be done when the letters are converted to numbers
    if len(bar)!=0 and 16%len(bar)==0:
        new_bar = ""
        time_value = 16//len(bar)
        for char in bar:
            new_bar+= char*time_value
        bar = new_bar
    return bar

In [39]:
test_tune = tunes_clean.sample(10)
for tune in test_tune['abc']:
    unfold_abc(tune)

E2GABGAB|EGBdc2A2|1cBceBABd|ABAGF2GF|2cBcedcdg|fefde4|gagfefgf|gagfdefe|fafedefd|1e2edB4|2efede4|
f|edcBAEE/E/E|AEFDEA,A,/A,/A,|
g|dBABGGGg|dBAGBBBg|dBABGGGe|dBgBAAAg|dBABGGGg|dBAGBBBe|degdegde|BdABGGG|g|e2dedBgd|e2deBBBg|e2dedBgd|e2dBAAAg|e2dedBgd|e2deBBBe|degdegde|BdABGGG|g|GGGBGGGB|GBAcBdBg|GGGBGGGe|dBgBAAAg|GGGBGGGB|GBAcBdBg|egdedeBd|BdABGGG|g|dBBdecce|dBgdBBBg|dBBdecce|dBgBAAAg|dBBdecce|dBgdBBBe|degdegde|BdABGGG|
|B2ABBDGB|c2BcEccE|BDGBBdcd|ADFAAdcA|B2ABBDGB|c2BcEccE|BDGBADFA|G4G4|d2BdGBBD|c2BcEccE|BDGBBdcd|ADFAAdcA|d2BdGBBD|c2BcEccE|BDGBADFA|G4G4|BDD2DBBD|cEE2Ecdc|BDD2Gdcd|ADFAAdcA|BDD2DBBD|cEE2Ecdc|BDGBADFA|G4G4|d2d2BcdB|cEEcEGcA|BDD2Gdcd|ADFAAdcA|d2d2BcdB|cEEcEGcA|BDGBADFA|G4G4|BDDBDDBD|cEEcEEcE|BDDBDDBD|ADBDcDdc|BDDBDDBD|cEEcEEcE|BDGBADFA|G4G4|
:2/4d/g/|ed/g/ed/g/|ed-dG|GG/G/AG|E2-EA/B/|c/B/A/B/cA/B/|c/B/A/B/cF/E/|De|Bd-dd/g/|ed/g/ed/g/|ed-dG|GG/G/A|E2-EA/B/|c/B/A/B/cA/B/|c/B/A/B/cF/E/|DD/D/EF/F/|GD/G/FE/4F/4E/|D2-D|G/A/|B/A/B/A/BD/D/|EE-EA|c/d/c/d/cA|FF/F/FG|AF/4G/4F/ED|DcBA|